# 가상환경 torch_cuda_yolo_env

# 추론 : 배포한 공유모델로 객체감지

In [1]:
import torch
print("torch:", torch.__version__, "cuda:", torch.version.cuda, "is_available:", torch.cuda.is_available())

torch: 2.6.0+cu124 cuda: 12.4 is_available: True


In [ ]:
import os
from dotenv import load_dotenv
loaded = load_dotenv()
# os.getenv("ROBOFLOW_API_KEY")

In [4]:
from inference_sdk import InferenceHTTPClient
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=os.getenv("ROBOFLOW_API_KEY")
)
CLIENT

In [7]:
result = CLIENT.infer('sample.png',model_id='rock-paper-scissors-sxsw/14')
result

{'inference_id': '8c0b7722-a77a-40e6-b40d-97bf4758541b',
 'time': 0.04968818800080044,
 'image': {'width': 580, 'height': 614},
 'predictions': [{'x': 129.5,
   'y': 119.0,
   'width': 257.0,
   'height': 96.0,
   'confidence': 0.8729892373085022,
   'class': 'Scissors',
   'class_id': 2,
   'detection_id': '855188de-9042-456f-b121-6e28e64fcfd2'},
  {'x': 117.0,
   'y': 316.5,
   'width': 230.0,
   'height': 123.0,
   'confidence': 0.7074464559555054,
   'class': 'Paper',
   'class_id': 0,
   'detection_id': 'cd046009-796c-40e2-8623-befeb1ecf59e'},
  {'x': 128.0,
   'y': 502.5,
   'width': 254.0,
   'height': 109.0,
   'confidence': 0.4878521263599396,
   'class': 'Paper',
   'class_id': 0,
   'detection_id': 'cce31e42-802b-4d66-8e8a-d5f3dc30775c'}]}

In [9]:
import cv2
img= cv2.imread('sample.png')

for pred in result['predictions']:
    x = pred['x']
    y = pred['y']
    width = pred['width']
    height = pred['height']
    conf = pred['confidence']
    obj_class = pred['class']
    
    x1, y1 = int(x-width/2), int(y-height/2)
    x2, y2 = int(x+width/2), int(y+height/2)
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,0,255), 2)
    cv2.putText(img, f'{obj_class} {conf:.4f}', (x1,y1), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255))
    
    print(x,y,width,height,conf,obj_class)
    
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

129.5 119.0 257.0 96.0 0.8729892373085022 Scissors
117.0 316.5 230.0 123.0 0.7074464559555054 Paper
128.0 502.5 254.0 109.0 0.4878521263599396 Paper


In [30]:
result = CLIENT.infer('mnist/000001.jpg', model_id='numbers-xnrog/2')
result

{'inference_id': 'ecc8a356-2ed8-4f6d-af9c-1d36161eb480',
 'time': 0.10049987599995802,
 'image': {'width': 416, 'height': 416},
 'predictions': [{'x': 140.0,
   'y': 209.0,
   'width': 24.0,
   'height': 32.0,
   'confidence': 0.6804451942443848,
   'class': '8',
   'class_id': 8,
   'detection_id': '6724ad75-0a69-4cfe-b4e5-80af5e20713f'},
  {'x': 192.5,
   'y': 220.0,
   'width': 13.0,
   'height': 20.0,
   'confidence': 0.6476726531982422,
   'class': '8',
   'class_id': 8,
   'detection_id': 'ce6f0d03-fd9d-45e1-8ea2-b9b78aefbd9c'}]}

# model 가져오기

In [37]:
from roboflow import Roboflow
import cv2
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace().project('numbers-yt0h2')
numbers_model = project.version(2).model
numbers_model

loading Roboflow workspace...
loading Roboflow project...


In [38]:
result = numbers_model.predict('mnist/000001.jpg', confidence=40, overlap=30)
result_json = result.json()
print(result_json)

{'predictions': [{'x': 156, 'y': 236, 'width': 49, 'height': 41, 'confidence': 0.5040796995162964, 'class': '2', 'class_id': 2, 'detection_id': '062cc7ea-8d9b-4264-90ce-ebbff91b5171', 'image_path': 'mnist/000001.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '416', 'height': '416'}}


In [39]:
img= cv2.imread('mnist/000001.jpg')
for pred in result_json['predictions']:
    x = pred['x']
    y = pred['y']
    width = pred['width']
    height = pred['height']
    conf = pred['confidence']
    obj_class = pred['class']
    
    x1, y1 = int(x-width/2), int(y-height/2)
    x2, y2 = int(x+width/2), int(y+height/2)
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,0,255), 2)
    cv2.putText(img, f'{obj_class} {conf:.4f}', (x1,y1), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255))
    
    print(x,y,width,height,conf,obj_class)
    
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

156 236 49 41 0.5040796995162964 2
